# Step 1: Make lambda layers, lambda function, and execute function

#### Firtst, lets create a few sample files in s3

In [ ]:
import boto3
import numpy
import pandas as pd
from io import StringIO

s3 = boto3.resource('s3')
lambda_c = boto3.client('lambda')

sample_count = 10000000 #enter sample count
sample_file_s3 = 'sagemaker-cfn' #enter sample file s3 bucket name
bucket_layer = "lambda-layer-for-python" # add your bucket name and needs to be same region as the lambda function
sample_file_s3_name = str(sample_count) + '-sample.csv' # modify if needed
sample_file_s3_name_out = str(sample_count) + '-sample-out' # modify if needed

df = pd.DataFrame(numpy.random.randint(12000,15000,size=(sample_count, 2)).astype("float")/1000)
out = StringIO()
pd.DataFrame({'0':df[0],'1':df[1]}).to_csv(out, header=False, index=False)
result = out.getvalue()

#bucket = s3.Bucket(sample_file_s3)
s3.Object(sample_file_s3, sample_file_s3_name).put(Body=result) # Ensure status code is 200

Install lambda-layer using pip. Ensure kernel is conda_python3

In [ ]:
!pip install lambda-layer

Provide layer name, version and the layer to be installed via pip

In [ ]:
layer_name="numpy"
layer_version="0.0.1"
layer="numpy"

In [ ]:
# creating env vairables
%env layer_name=$layer_name
%env layer_version=$layer_version
%env layer=$layer

# creating config file
!printf "[[layers]]\n name = \"$layer_name\"\n version = \"$layer_version\"\n packages = [\n     '$layer'\n]" > req.txt
!cat req.txt

In [ ]:
output=!lambda-layer package -c req.txt |grep "Copying" |awk '{print $4}'
layer_output = output[0]
layer_output_name = layer_output.split("/")[-1]
print(f"Created {layer_output_name} at {layer_output}")

### Your lambda layer is created. But you dont need it. Lets download aws-data-wrangler.

via below link:

https://github.com/awslabs/aws-data-wrangler/releases

In [ ]:
#check version
wr_path = "https://github.com/awslabs/aws-data-wrangler/releases/download/0.3.2/awswrangler-layer-0.3.2-py3.8.zip"
wr_name = wr_path.split("/")[-1]
wr_name_short = wr_path.split("/")[-1].split("-")[0]

%env wr_path=$wr_path
%env wr_name=$wr_name
%env wr_name_short=$wr_name_short

!wget $wr_path

In [ ]:


get_region=!aws configure get region
region = get_region[0]
%env region=$region
%env bucket_layer=$bucket_layer

s3_path1 = s3.Bucket(bucket_layer).upload_file(layer_output, layer_output_name)
s3_path2 = s3.Bucket(bucket_layer).upload_file(wr_name, wr_name)

for files in s3.Bucket(bucket_layer).objects.all():
    print(f"https://{files.bucket_name}.s3.{region}.amazonaws.com/{files.key}")

#### Run the below command to publish the lambda layer.  Copy the ouput and paste into cfn-lambda.template like described below.

```
        "AWSWRLambda": {
            "Type": "AWS::Lambda::Function",
                ........
                "Timeout": 900,
                "Handler": "lambda_function.lambda_handler",
                "Runtime": "python3.8",
                "MemorySize": 3000,
                "Layers": ["COPY ARN HERE"] <---
            }
        }
```

In [ ]:
!echo "Copy the below output to cfn-lambda.template"
!aws lambda publish-layer-version \
    --layer-name $wr_name_short \
    --description $wr_name \
    --license-info "MIT" \
    --content S3Bucket=$BUCKET_layer,S3Key=$wr_name \
    --compatible-runtimes python3.8 \
    --region $region \
    --query LayerVersionArn
layer_versions=!aws lambda list-layer-versions --layer-name $wr_name_short --query LayerVersions[0].[Version][0]
layer_version=layer_versions[0]
%env layer_version=$layer_version
layer_arns=!aws lambda get-layer-version --layer-name $wr_name_short --version-number $layer_version --query LayerVersionArn
layer_arn=layer_arns[0].replace('"','')
%env layer_arn=$layer_arn

In [ ]:
!aws lambda get-layer-version --layer-name $wr_name_short --version-number $layer_version --query LayerVersionArn


#### Copy python code to s3

In [ ]:
BUCKET_function = "lambda-function-for-python" # add your bucket name and needs to be in region of the lambda function
%env BUCKET_function=$BUCKET_function
!rm lambda_function.zip
!zip lambda_function.zip lambda_function.py
s3_path = s3.Bucket(BUCKET_function).upload_file("lambda_function.zip", "lambda_function.zip")

#### Add bucket details to cfn-lambda.template

```
        "AWSWRLambda": {
            "Type": "AWS::Lambda::Function",
            ........
            "Properties": {
                "Code": {
                    "S3Bucket": "ENTER BUCKET NAME", <----
                    "S3Key": "lambda-function.zip"
                },
             .......
            }
        }
```

### Lambda layer is now published and we can make the lambda function

In [ ]:
!aws cloudformation create-stack --template-body file://cfn-lambda.template --stack-name cfn-lambda --parameters ParameterKey=bucket,ParameterValue=$BUCKET_function ParameterKey=layer,ParameterValue=$layer_arn --capabilities CAPABILITY_IAM

#### keep running the below command until an output with "CREATE_COMPLETE" is generated. Around 2 min.  Copy the arn to the below command.

In [ ]:
!aws cloudformation describe-stacks --stack-name cfn-lambda --query Stacks[*][StackName,StackStatus,Outputs[*].[OutputValue]]
lambda_arns=!aws cloudformation describe-stacks --stack-name cfn-lambda --query Stacks[*][StackName,StackStatus,Outputs[*].[OutputValue]] | grep arn | sed -e 's/^\s*//' -e '/^$/d'
lambda_arn = lambda_arns[0].replace('"',"")
%env lambda_arn=$lambda_arn

In [ ]:
payload = '{ "input": "s3://sagemaker-cfn/' + sample_file_s3_name + '", "output":"s3://sagemaker-cfn/' + sample_file_s3_name_out + '" }'
response = lambda_c.invoke(
    FunctionName=lambda_arn,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=payload
)

print(response['StatusCode'])
print(response['Payload'].read().decode())